In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.common.exceptions import TimeoutException
from time import sleep
import os
import xmltodict
import json
import re

In [2]:
chrome_driver = '/Users/nicholasdevito/Dropbox/Python projects/ANZCTR/chromedriver'
download_path = '/Users/nicholasdevito/Desktop/CRiS Test'

In [ ]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : download_path,
        'disk-cache-size': 4096}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=chrome_driver, options=chromeOptions)
driver.get('http://cris.nih.go.kr/cris/en/search/basic_search.jsp')
driver.find_element_by_xpath("//input[@alt='Search']").click()
driver.find_element_by_xpath("//img[@alt='Download']").click()
sleep(2)
handles = driver.window_handles
driver.switch_to.window(handles[-1])
select = Select(driver.find_element_by_id('data_dtype'))
select.select_by_value('xml')
driver.find_element_by_xpath("//img[@alt='confirm']").click()

#this is untested for this code at the moment because downloading takes forever
dl_check = 0
while dl_check == 0:
    count = 0
    for file in os.listdir(download_path):
        if file.endswith(".crdownload") or len(os.listdir(download_path)) == 0:
            count = count+1
    if count == 0:
        dl_check = 1
        driver.quit()
    else:
        dl_check = 0
        sleep(3)

In [3]:
print(os.listdir(download_path))
file_name = ''
for file in os.listdir(download_path):
    if file.endswith('.xml'):
        file_name = file
print(file_name)

['cris_research_2019-02-11_en.xml', '.DS_Store']
cris_research_2019-02-11_en.xml


In [4]:
#there's a problematic set of characters (&#1;) in the xml making it not valid. This should clean it up:

xmlstring = open(download_path + '/' + file_name, 'r').read()
xml_fixed = re.sub(r'&#1;', '', xmlstring)

In [32]:
cris_trials = []
cris_json = []

trials = xmltodict.parse(xml_fixed)
for trial in trials['trials']['trial']:
    cris_trials.append(str(json.dumps(trial)))
    cris_json.append(json.dumps(trial))


In [22]:
print(cris_trials[62])

{"main": {"trial_id": "KCT0003420", "reg_name": "CRIS", "date_registration": "11/01/2019", "data_update": "23/11/2018", "protocol_id": "HYI-15-197-2", "primary_sponsor": {"prim_sponsor_en": "Hanyang University Seoul Hospital"}, "scientific_title": {"sci_title_en": "Effect of Computer-Assisted Cognitive Rehabilitation in Unilateral Spatial Neglect Patients with Brain Damage"}, "scientific_acronym": "Computer-Assisted Cognitive Rehabilitation(CACR), Conventional Rehabilitation(CR)", "date_enrolment": "04/01/2016", "date_completion": null, "recruitment_status": "Completed", "study_type": "Interventional Study", "phase": "Not applicable", "i_code": "Others", "inclusion_criteria": {"inc_criteria_en": "The inclusion criteria were as follows: patients who were diagnosed with brain damage or brain metabolic diseases by computerized tomography or magnetic resonance imaging; patients with suggested or indicated (moderate/severe) unilateral spatial neglect who under responding less than normal ra

In [33]:
test = json.loads(cris_trials[62])
print(test)

{'main': {'trial_id': 'KCT0003420', 'reg_name': 'CRIS', 'date_registration': '11/01/2019', 'data_update': '23/11/2018', 'protocol_id': 'HYI-15-197-2', 'primary_sponsor': {'prim_sponsor_en': 'Hanyang University Seoul Hospital'}, 'scientific_title': {'sci_title_en': 'Effect of Computer-Assisted Cognitive Rehabilitation in Unilateral Spatial Neglect Patients with Brain Damage'}, 'scientific_acronym': 'Computer-Assisted Cognitive Rehabilitation(CACR), Conventional Rehabilitation(CR)', 'date_enrolment': '04/01/2016', 'date_completion': None, 'recruitment_status': 'Completed', 'study_type': 'Interventional Study', 'phase': 'Not applicable', 'i_code': 'Others', 'inclusion_criteria': {'inc_criteria_en': 'The inclusion criteria were as follows: patients who were diagnosed with brain damage or brain metabolic diseases by computerized tomography or magnetic resonance imaging; patients with suggested or indicated (moderate/severe) unilateral spatial neglect who under responding less than normal ra

In [38]:
test_new = test['main']['trial_id']
print(test_new)

KCT0003420


In [ ]:
#creates a csv with no headers and a single column with all trials in json in their own row.

import csv
with open('test.csv','w', newline = '')as test_csv:
    writer=csv.writer(test_csv)
    for val in cris_trials:
        writer.writerow([val])

In [ ]:
#next up:
#1. The CRIS XML doesn't include anything that tells us about results status
#but we can automate getting another data download of just trials with results
#then we can get a list of those trial ids, and create indicator variables with them